# See a Summary of All Activities

In [8]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import ConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, StepsActivities, ActivityLaps, ActivityRecords
from idbutils.list_and_dict import list_not_none


db_params_dict = ConfigManager.get_db_params()

garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]

def __report_sport(sport_col, sport):
    records = Activities.row_count(garmin_act_db, sport_col, sport)
    if records > 0:
        sport_title = sport.title().replace('_', ' ')
        total_distance = Activities.get_col_sum_for_value(garmin_act_db, Activities.distance, sport_col, sport)
        if total_distance is None:
            total_distance = 0
            average_distance = 0
        else:
            average_distance = total_distance / records
        return [sport_title, records, total_distance, average_distance]

doc = snakemd.new_doc("activities")

doc.add_header("Activities Report")
doc.add_paragraph("Analysis of all activities in the database.")

doc.add_table(
    ['Type', 'Count'],
    [
        ["Total activities", Activities.row_count(garmin_act_db)],
        ["Total Lap records", ActivityLaps.row_count(garmin_act_db)],
        ["Activity records", ActivityRecords.row_count(garmin_act_db)],
        ["Fitness activities", Activities.row_count(garmin_act_db, Activities.type, 'fitness')],
        ["Recreation activities", Activities.row_count(garmin_act_db, Activities.type, 'recreation')]
    ])

years = Activities.get_years(garmin_act_db)
years.sort()
doc.add_paragraph(f"Years with activities: {len(years)}: {years}")
sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sport))
doc.add_paragraph(f"Sports: {', '.join(sports)}")
sub_sports = list_not_none(Activities.get_col_distinct(garmin_act_db, Activities.sub_sport))
doc.add_paragraph(f"SubSports: {', '.join(sub_sports)}")

sports_stats = []
for sport_name in [sport.name for sport in fitfile.Sport]:
    sport_stat = __report_sport(Activities.sport, sport_name)
    if sport_stat:
        sports_stats.append(sport_stat)
doc.add_header("Sport Type Statistics", 3)
doc.add_table(['Sport', 'Total Activities', f'Total Distance ({distance_units})', f"Average Distance ({distance_units})"], sports_stats)

def __format_activity(activity):
    if activity:
        if activity.is_steps_activity():
            steps_activity = StepsActivities.get(garmin_act_db, activity.activity_id)
            return [activity.activity_id, activity.name, activity.type, activity.sport, activity.distance, activity.elapsed_time, activity.avg_speed, steps_activity.avg_pace, activity.calories]
        return [activity.activity_id, activity.name, activity.type, activity.sport, activity.distance, activity.elapsed_time, activity.avg_speed, '', activity.calories]
    return ['', '', '', '', '', '', '', '', '']

activities = Activities.get_latest(garmin_act_db, 10)
rows = [__format_activity(activity) for activity in activities]
doc.add_header("Last Ten Activities", 3)
doc.add_table(['Id', 'Name', 'Type', 'Sport', f'Distance ({distance_units})', 'Elapsed Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})', f'Pace ({unit_strings[fitfile.units.UnitTypes.pace]})', 'Calories'], rows)

doc.add_header("Interesting Activities", 3)
doc.add_table(['What', 'Id', 'Name', 'Type', 'Sport', f'Distance ({distance_units})', 'Elapsed Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})', f'Pace ({unit_strings[fitfile.units.UnitTypes.pace]})', 'Calories'], [
    ['Latest Walk'] + __format_activity(Activities.get_latest_by_sport(garmin_act_db, fitfile.Sport.walking)),
    ['Fastest Walk'] + __format_activity(Activities.get_fastest_by_sport(garmin_act_db, fitfile.Sport.walking)),
    ['Slowest Walk'] + __format_activity(Activities.get_slowest_by_sport(garmin_act_db, fitfile.Sport.walking)),
    ['Longest Walk'] + __format_activity(Activities.get_longest_by_sport(garmin_act_db, fitfile.Sport.walking)),
    ['Latest Run'] + __format_activity(Activities.get_latest_by_sport(garmin_act_db, fitfile.Sport.running)),
    ['Fastest Run'] + __format_activity(Activities.get_fastest_by_sport(garmin_act_db, fitfile.Sport.running)),
    ['Slowest Run'] + __format_activity(Activities.get_slowest_by_sport(garmin_act_db, fitfile.Sport.running)),
    ['Longest Run'] + __format_activity(Activities.get_longest_by_sport(garmin_act_db, fitfile.Sport.running)),
    ['Latest Hike'] + __format_activity(Activities.get_latest_by_sport(garmin_act_db, fitfile.Sport.hiking)),
    ['Fastest Hike'] + __format_activity(Activities.get_fastest_by_sport(garmin_act_db, fitfile.Sport.hiking)),
    ['Slowest Hike'] + __format_activity(Activities.get_slowest_by_sport(garmin_act_db, fitfile.Sport.hiking)),
    ['Longest Hike'] + __format_activity(Activities.get_longest_by_sport(garmin_act_db, fitfile.Sport.hiking)),
    ['Latest Paddle'] + __format_activity(Activities.get_latest_by_sport(garmin_act_db, fitfile.Sport.paddling)),
    ['Fastest Paddle'] + __format_activity(Activities.get_fastest_by_sport(garmin_act_db, fitfile.Sport.paddling)),
    ['Slowest Paddle'] + __format_activity(Activities.get_slowest_by_sport(garmin_act_db, fitfile.Sport.paddling)),
    ['Longest Paddle'] + __format_activity(Activities.get_longest_by_sport(garmin_act_db, fitfile.Sport.paddling))
])

display(Markdown(str(doc)))

# Activities Report

Analysis of all activities in the database.

| Type                  | Count   |
| --------------------- | ------- |
| Total activities      | 929     |
| Total Lap records     | 2744    |
| Activity records      | 2280550 |
| Fitness activities    | 486     |
| Recreation activities | 418     |

Years with activities: 13: [2007, 2009, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

Sports: hiking, walking, cycling, fitness_equipment, running, alpine_skiing, paddling, inline_skating, stand_up_paddleboarding, snowshoeing

SubSports: generic, mountain, elliptical, treadmill, indoor_walking, strength_training

### Sport Type Statistics

| Sport                   | Total Activities | Total Distance (miles) | Average Distance (miles) |
| ----------------------- | ---------------- | ---------------------- | ------------------------ |
| Running                 | 147              | 447.78114498996814     | 3.0461302380269943       |
| Cycling                 | 81               | 592.6610357484641      | 7.316802910474865        |
| Fitness Equipment       | 216              | 74397.38548900632      | 344.43234022688114       |
| Walking                 | 313              | 838.910933191712       | 2.6802266236156935       |
| Alpine Skiing           | 9                | 58.578023826927996     | 6.508669314103111        |
| Hiking                  | 104              | 283.8749302245921      | 2.729566636774924        |
| Paddling                | 47               | 163.21641195512        | 3.472689616066383        |
| Inline Skating          | 4                | 24.769614512496        | 6.192403628124           |
| Snowshoeing             | 4                | 5.287881339424         | 1.321970334856           |
| Stand Up Paddleboarding | 4                | 10.20117526464         | 2.55029381616            |

### Last Ten Activities

| Id         | Name                              | Type       | Sport   | Distance (miles)   | Elapsed Time    | Speed (mph)        | Pace (per mile) | Calories |
| ---------- | --------------------------------- | ---------- | ------- | ------------------ | --------------- | ------------------ | --------------- | -------- |
| 8090181358 | Cromesett Walk                    | recreation | walking | 4.783818808272     | 01:45:14.524000 | 2.72682494346714   | 00:22:00.216763 | 453      |
| 7951619327 | Great Neck Conservation Area Walk | recreation | walking | 1.316735282496     | 00:28:38.748000 | 2.758142177598953  | 00:21:45.226405 | 138      |
| 7945766272 | Cromesett Dog Walk                | recreation | walking | 2.510395571408     | 00:45:17.977000 | 3.324086765868187  | 00:18:03.004221 | 254      |
| 7871721178 | Running of the Pilgrims           | race       | running | 3.2205420747519997 | 00:29:58.072000 | 6.4490862576026915 | 00:09:18.218615 | 328      |
| 7813800588 | CC Canal Run                      | fitness    | running | 1.425444173936     | 00:18:11.141000 | 6.216445242132187  | 00:09:39.109098 | 146      |
| 7804071899 | Cromesett 2 Mile Run              | fitness    | running | 2.0085886177120003 | 00:18:40.711000 | 6.451323564798355  | 00:09:18.025026 | 188      |
| 7782804092 | Canal Dog Walk                    | recreation | walking | 2.8061744763200003 | 00:52:56.281000 | 3.1809231050653457 | 00:18:51.746943 | 265      |
| 7776300546 | Cromesett Run                     | fitness    | running | 2.731398666112     | 00:26:26.491000 | 6.198549451202393  | 00:09:40.781041 | 268      |
| 7749731018 | Sandwich Canal Walk               | recreation | walking | 4.90625378952      | 01:43:22.157000 | 2.847619532266617  | 00:21:04.213832 | 481      |
| 7736609401 | Canal Dog Walk                    | recreation | walking | 3.1936864114879997 | 01:02:46.968000 | 3.051180620800972  | 00:19:39.871154 | 322      |

### Interesting Activities

| What           | Id         | Name                                       | Type          | Sport    | Distance (miles)   | Elapsed Time    | Speed (mph)        | Pace (per mile) | Calories           |
| -------------- | ---------- | ------------------------------------------ | ------------- | -------- | ------------------ | --------------- | ------------------ | --------------- | ------------------ |
| Latest Walk    | 8090181358 | Cromesett Walk                             | recreation    | walking  | 4.783818808272     | 01:45:14.524000 | 2.72682494346714   | 00:22:00.216763 | 453                |
| Fastest Walk   | 4389758850 | UCT: Holliston Center 5K                   | fitness       | walking  | 3.1870812356319997 | 00:50:10.087000 | 3.811738944        | 00:15:44.450828 | 350                |
| Slowest Walk   | 5366582102 | Hopkinton Walking                          | uncategorized | walking  | 1.201178880432     | 01:06:09.936000 | 1.089387806400299  | 00:55:04.608312 | 378                |
| Longest Walk   | 1668042652 | UCT: Holliston Center to Milford Roundtrip | fitness       | walking  | 8.335905914208     | 02:43:15.106000 | 3.661864232        | 00:16:23.105809 | 956                |
| Latest Run     | 7871721178 | Running of the Pilgrims                    | race          | running  | 3.2205420747519997 | 00:29:58.072000 | 6.4490862576026915 | 00:09:18.218615 | 328                |
| Fastest Run    | 1967470443 | Live4Evan 5K                               | race          | running  | 3.151650649808     | 00:25:45.853000 | 7.339387016000001  | 00:08:10.504179 | 341                |
| Slowest Run    | 4085785632 | Run/Walk with Izzy                         | fitness       | running  | 3.4218352749919996 | 00:52:07.279000 | 3.9392442960000005 | 00:15:13.880869 | 490                |
| Longest Run    | 2875915728 | Budapest Running                           | fitness       | running  | 5.078783716912     | 00:51:26.124000 | 5.939065080000001  | 00:10:06.156011 | 608                |
| Latest Hike    | 7652038710 | Champney Falls Hike                        | recreation    | hiking   | 3.611620680608     | 02:03:10.479000 | 1.758231725866318  | 00:34:07.511683 | 538                |
| Fastest Hike   | 1683547910 | USF Hike                                   | fitness       | hiking   | 3.091862312944     | 00:54:42.555000 | 3.391194976        | 00:17:41.572698 | 329                |
| Slowest Hike   | 1603226005 | Whitehall Hike                             | recreation    | hiking   | 1.200004488864     | 01:40:05        | 0.682898441656655  | 01:27:51.647701 | 352.02695929852365 |
| Longest Hike   | 2062164431 | Franconia Ridge Loop Hike                  | recreation    | hiking   | 9.124438394432     | 06:48:47.508000 | 1.339924664        | 00:44:46.718214 | 2262               |
| Latest Paddle  | 7477222158 | Cove Paddle                                | recreation    | paddling | 1.3546637805439998 | 00:40:52.537000 | 1.9884670087831497 |                 | 167                |
| Fastest Paddle | 7229419789 | First Mark's Cove Kayak                    | recreation    | paddling | 1.747960681584     | 00:42:33.746000 | 2.465103467733383  |                 | 188                |
| Slowest Paddle | 5344941805 | Early Morning Whitehall Kayak              | recreation    | paddling | 3.147829216928     | 03:04:25.684000 | 1.0245166975998878 |                 | 706                |
| Longest Paddle | 2168721083 | East Branch of the Westport River Kayak    | recreation    | paddling | 7.001778451824     | 03:29:22.685000 | 2.006531592        |                 | 798                |